In [ ]:
import os
from os.path import join as ojoin
sim_dir = ojoin('..', 'simulation_results') 

In [ ]:
def extract_coverage(fpath):
    total_coverage = 0
    with open(fpath) as f:
        for line in f:
            line = line.strip()
            if not line: continue
            total_coverage += float(line)
    return total_coverage

In [ ]:
best_coverage = [extract_coverage(ojoin(sim_dir, 'tim', 'output_%d.txt'%i)) for i in range(20)]

In [ ]:
pof_dict = dict()

for attribute in ('age', 'gender', 'ethnicity'):
    for method in ('equality', 'equity', 'group_maximin', 'dc'):
        output_dir = ojoin(sim_dir, 'mip', method, attribute)
        pofs = []
        for graph_number in range(20):
            output_file = ojoin(output_dir, 'output_%d.txt'%graph_number)
            coverage = extract_coverage(output_file)
            if coverage > 0:
                pof = best_coverage[graph_number] / coverage
                pofs.append(pof)
        pof_dict[(method, attribute)] = pofs

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np



def draw(values, labels, colors, title, ylabel, fpath):
    plt.figure(figsize=(3, 2))
    means = [np.mean(v) for v in values]
    stds = [np.std(v) for v in values]
    xpos = range(len(labels))

    bars = plt.bar(xpos, means, yerr=stds, width=0.5, align='center', capsize=4)
    for i in xpos:
        bars[i].set_color(colors[i])
    _ = plt.xticks(xpos, labels, rotation=0, fontsize=10)
    _ = plt.ylim([0.9, 1.2])
    _ = plt.ylabel(ylabel, fontsize=10)
    _ = plt.title(title, fontsize=12)
    _ = plt.savefig(fpath, bbox_inches='tight')
    

In [ ]:
labels = ['equality', 'equity', 'group\nmaximin', 'diversity']
colors = ['#8fffa0'] * 4

for attribute in ('age', 'gender', 'ethnicity'):
    values = [np.array(pof_dict[(method, attribute)]) 
              for method in ('equality', 'equity', 'group_maximin', 'dc')]

    title = attribute[0].upper() + attribute[1:]
    fpath = 'images/mip_pof/%s.pdf'%attribute
    draw(values, labels, colors, title, 'Price of Fairness', fpath)